In [6]:
# pip install selenium pandas pymysql sqlalchemy openpyxl

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import re
from pathlib import Path
from datetime import datetime

# ─────────────────────────────────────────────────────────
# 설정
# ─────────────────────────────────────────────────────────
BASE = "http://office.anyfive.com/home/"

# 입력 파일
NEED_CRAWLING_JSON = "need_crawling.json"
CMDS_INPUT = r"C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\2020\documents_2020_01.cmds"
CMDS_OUTPUT = "documents_2020_actTimefixed.cmds"

# 크롤링 결과 임시 저장 (에러 복구용)
CRAWL_RESULT_JSON = "crawl_result_temp.json"

# ─────────────────────────────────────────────────────────
# 유틸 함수 (기존 코드에서 가져옴)
# ─────────────────────────────────────────────────────────
def wait_click_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))

def wait_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))

def click_in_all_frames(css_list=None, xp_list=None, text_contains=None):
    """여러 프레임을 훑으면서 클릭 시도"""
    driver.switch_to.default_content()
    
    # 최상위
    if css_list:
        for css in css_list:
            els = driver.find_elements(By.CSS_SELECTOR, css)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    if xp_list:
        for xp in xp_list:
            els = driver.find_elements(By.XPATH, xp)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    
    # 프레임 내부
    frames = driver.find_elements(By.CSS_SELECTOR, "iframe, frame")
    for fr in frames:
        driver.switch_to.default_content()
        driver.switch_to.frame(fr)
        if css_list:
            for css in css_list:
                els = driver.find_elements(By.CSS_SELECTOR, css)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
        if xp_list:
            for xp in xp_list:
                els = driver.find_elements(By.XPATH, xp)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
    
    driver.switch_to.default_content()
    return False

def switch_into_frame_having(selector_css, max_retries=3, wait_time=2):
    """selector_css 요소가 보이는 프레임으로 전환"""
    for attempt in range(max_retries):
        driver.switch_to.default_content()
        time.sleep(wait_time)
        
        # 최상위에서 찾기
        if driver.find_elements(By.CSS_SELECTOR, selector_css):
            print(f"✓ 요소 발견 (최상위)")
            return True
        
        # 프레임 내부 탐색
        frames = driver.find_elements(By.CSS_SELECTOR, "iframe, frame")
        for idx, fr in enumerate(frames):
            try:
                driver.switch_to.default_content()
                driver.switch_to.frame(fr)
                time.sleep(0.3)
                
                if driver.find_elements(By.CSS_SELECTOR, selector_css):
                    print(f"✓ 요소 발견 (프레임 {idx})")
                    return True
            except:
                continue
        
        driver.switch_to.default_content()
        
        if attempt < max_retries - 1:
            print(f"⚠️ 요소 미발견, 재시도 중... ({attempt+1}/{max_retries})")
    
    return False

def set_kendo_dropdown_by_text(wrapper_css, item_text, sec=15):
    """Kendo DropDownList wrapper 클릭 → 텍스트로 선택"""
    w = wait_click_css(wrapper_css, sec)
    listbox_id = w.get_attribute("aria-owns")
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", w)
    driver.execute_script("arguments[0].click();", w)
    time.sleep(0.5)
    xp = f"//ul[@id='{listbox_id}']//li[contains(normalize-space(.), '{item_text}')]"
    el = WebDriverWait(driver, sec).until(EC.element_to_be_clickable((By.XPATH, xp)))
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    driver.execute_script("arguments[0].click();", el)

def set_input_value(css, value):
    """입력 필드에 값 설정"""
    el = wait_css(css, 10)
    driver.execute_script(
        "arguments[0].value = arguments[1];"
        "arguments[0].dispatchEvent(new Event('input',{bubbles:true}));"
        "arguments[0].dispatchEvent(new Event('change',{bubbles:true}));",
        el, value
    )
    return el

def norm_space(s):
    """공백 정규화"""
    if not s:
        return ""
    s = s.replace('\xa0', ' ')
    s = re.sub(r'[\t\r\n]+', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

def parse_datetime_to_timestamp(date_str):
    """날짜시간 문자열을 밀리초 timestamp로 변환"""
    # 예: "2019-12-30 22:04:45" → 1577712285000
    try:
        dt = datetime.strptime(date_str.strip(), "%Y-%m-%d %H:%M:%S")
        return int(dt.timestamp() * 1000)
    except Exception as e:
        print(f"⚠️ 날짜 파싱 실패: {date_str} - {e}")
        return None

def wait_grid_stable(timeout=10):
    """그리드 안정화 대기"""
    try:
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
        )
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table[role='grid'] tbody tr"))
        )
        time.sleep(1)
        return True
    except:
        return False

# ─────────────────────────────────────────────────────────
# 페이징 함수
# ─────────────────────────────────────────────────────────
def has_next_page():
    """다음 페이지 존재 여부"""
    try:
        next_btns = driver.find_elements(By.CSS_SELECTOR, 
            "a.k-pager-nav[aria-label='다음 페이지로 이동']:not(.k-state-disabled)")
        return len(next_btns) > 0
    except:
        return False

def go_to_next_page():
    """다음 페이지로 이동"""
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR,
            "a.k-pager-nav[aria-label='다음 페이지로 이동']:not(.k-state-disabled)")
        
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)
        
        # 로딩 대기
        WebDriverWait(driver, 15).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
        )
        
        wait_grid_stable()
        return True
    except Exception as e:
        print(f"  ❌ 페이지 이동 실패: {e}")
        return False

# ─────────────────────────────────────────────────────────
# 1) need_crawling.json 읽기
# ─────────────────────────────────────────────────────────
print("="*70)
print("🚀 결재 시간 크롤링 시작")
print("="*70)

need_crawling = {}
try:
    with open(NEED_CRAWLING_JSON, 'r', encoding='utf-8') as f:
        need_crawling = json.load(f)
    print(f"✅ {NEED_CRAWLING_JSON} 로드 완료: {len(need_crawling)}개 문서")
except Exception as e:
    print(f"❌ {NEED_CRAWLING_JSON} 읽기 실패: {e}")
    exit(1)

# sourceId를 set으로 변환 (빠른 검색용)
target_source_ids = set(need_crawling.keys())
print(f"📋 크롤링 대상: {len(target_source_ids)}개 문서\n")

# 크롤링 결과 저장용
crawl_results = {}

# 기존 임시 결과 로드 (중단 후 재시작용)
if Path(CRAWL_RESULT_JSON).exists():
    try:
        with open(CRAWL_RESULT_JSON, 'r', encoding='utf-8') as f:
            crawl_results = json.load(f)
        print(f"✅ 기존 크롤링 결과 로드: {len(crawl_results)}개")
        # 이미 크롤링한 것은 제외
        target_source_ids -= set(crawl_results.keys())
        print(f"📋 남은 크롤링 대상: {len(target_source_ids)}개\n")
    except:
        pass

# ─────────────────────────────────────────────────────────
# 2) 브라우저 시작
# ─────────────────────────────────────────────────────────
opts = webdriver.ChromeOptions()
opts.add_argument("--start-maximized")
opts.add_argument("--disable-popup-blocking")
opts.add_argument("--disable-blink-features=AutomationControlled")
opts.add_experimental_option("excludeSwitches", ["enable-automation"])
opts.add_experimental_option('useAutomationExtension', False)
opts.add_argument("--disable-dev-shm-usage")
opts.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(), options=opts)

try:
    driver.get(BASE)
    time.sleep(1.2)

    # ─────────────────────────────────────────────────────────
    # 3) 로그인 대기
    # ─────────────────────────────────────────────────────────
    input("✋ 로그인 완료 후 Enter를 눌러주세요... ")

    # ─────────────────────────────────────────────────────────
    # 4) 전자결재 → 결재 문서관리
    # ─────────────────────────────────────────────────────────
    print("\n📂 전자결재 메뉴 진입 중...")
    click_in_all_frames(
        css_list=["a[href='/apr/'].left_menu", "a[href='/apr/']"],
        xp_list=["//a[@href='/apr/']"],
    )
    time.sleep(0.8)

    print("📂 결재 문서관리 진입 중...")
    click_in_all_frames(
        css_list=["li[onclick*='managementDoc']"],
        xp_list=["//*[@onclick and contains(.,'managementDoc')] | //*[contains(.,'결재 문서관리')]"],
    )
    time.sleep(1.0)

    # ─────────────────────────────────────────────────────────
    # 5) 필터 설정 (전체 조회)
    # ─────────────────────────────────────────────────────────
    print("\n⚙️ 필터 설정 중...")
    switch_into_frame_having("header, .tabConts_headerBx, #searchFrm, form#managementdocForm")

    # 상태 = 완료
    STATUS_WRAPPER = "span.k-dropdown[aria-owns='documentStatusFilter_listbox']"
    try:
        print("  - 상태: 완료")
        set_kendo_dropdown_by_text(STATUS_WRAPPER, "완료")
    except Exception:
        pass

    # 시작일/종료일 설정
    try:
        print("  - 기간: 2016-01-01 ~ 2019-12-31")
        set_input_value("#startDatePicker", "2016-01-01")
        set_input_value("#endDatePicker", "2019-12-31")
    except Exception:
        pass

    # 조회 버튼 클릭
    print("\n🔍 조회 버튼 클릭...")
    try:
        btn = wait_click_css("#documentSearch")
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
        driver.execute_script("arguments[0].click();", btn)
    except Exception:
        click_in_all_frames(
            xp_list=["//button[normalize-space()='조회']"],
        )
    time.sleep(2)

    # ─────────────────────────────────────────────────────────
    # 6) 페이징 순회하면서 크롤링
    # ─────────────────────────────────────────────────────────
    print("\n📊 그리드 데이터 크롤링 중...")

    GRID = "table[role='grid']"
    
    if not switch_into_frame_having(GRID):
        print("❌ 그리드를 찾을 수 없습니다!")
        driver.quit()
        exit(1)

    # 로딩 대기
    wait_grid_stable()

    found_count = 0
    error_count = 0
    page_num = 1

    while True:
        print(f"\n{'='*70}")
        print(f"📄 페이지 {page_num} 처리 중...")
        print(f"{'='*70}")
        
        # 현재 페이지 로딩 대기
        wait_grid_stable()

        # 현재 페이지의 체크박스 개수만 먼저 확인
        checkboxes = driver.find_elements(By.CSS_SELECTOR, "input[name='aprDoc_grid_ckbx']")
        total_in_page = len(checkboxes)
        print(f"  - 현재 페이지 문서 수: {total_in_page}개")
        
        # 인덱스 기반으로 순회 (매번 새로 찾기)
        processed_in_page = 0
        idx = 0
        
        while idx < total_in_page:
            # 매번 체크박스를 새로 찾기 (stale 방지)
            checkboxes = driver.find_elements(By.CSS_SELECTOR, "input[name='aprDoc_grid_ckbx']")
            
            if idx >= len(checkboxes):
                break
            
            cb = checkboxes[idx]
            source_id = cb.get_attribute("value")
            
            # 크롤링 대상이 아니면 다음으로
            if source_id not in target_source_ids:
                idx += 1
                continue

            print(f"\n  🎯 문서 발견: {source_id} (페이지 내 {idx+1}/{total_in_page})")
            
            try:
                # row 클릭 (상세 페이지 진입)
                tr = cb.find_element(By.XPATH, "./ancestor::tr")
                
                # 제목 링크 찾기
                title_link = tr.find_elements(By.CSS_SELECTOR, "a.grdTit")
                if not title_link:
                    print(f"    ⚠️ 제목 링크 없음, 스킵")
                    idx += 1
                    continue
                
                # 클릭
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", title_link[0])
                driver.execute_script("arguments[0].click();", title_link[0])
                time.sleep(2)
                
                # 상세 페이지로 전환 (새 프레임 찾기)
                if not switch_into_frame_having("tbody tr.F_12_gray, table"):
                    print(f"    ⚠️ 상세 페이지 로딩 실패")
                    driver.back()
                    time.sleep(1)
                    switch_into_frame_having(GRID)
                    idx += 1
                    continue
                
                # 결재선 테이블 파싱
                activities_data = []
                rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr.F_12_gray")
                
                for row_idx, row in enumerate(rows):
                    tds = row.find_elements(By.CSS_SELECTOR, "td")
                    if len(tds) < 6:
                        continue
                    
                    # 순서, 결재구분, 결재상태, 결재일시, 부서, 결재자
                    order = norm_space(tds[0].text)  # 1, 2, 3...
                    action_text = norm_space(tds[2].text)  # 기안, 승인, 합의 등
                    date_time = norm_space(tds[3].text)  # 2019-12-30 22:04:45
                    approver = norm_space(tds[5].text)  # 권재국 이사
                    
                    # 이름만 추출 (직급 제거)
                    name_match = re.match(r'([^\s]+)', approver)
                    name = name_match.group(1) if name_match else approver
                    
                    # action_type 매핑
                    action_type = None
                    if "기안" in action_text:
                        action_type = "DRAFT"
                    elif "승인" in action_text:
                        action_type = "APPROVAL"
                    elif "합의" in action_text:
                        action_type = "APPROVAL"  # 합의도 APPROVAL로 처리
                    
                    if date_time and action_type:
                        timestamp = parse_datetime_to_timestamp(date_time)
                        if timestamp:
                            activities_data.append({
                                "index": row_idx,
                                "name": name,
                                "action_type": action_type,
                                "datetime": date_time,
                                "timestamp": timestamp
                            })
                
                # 결과 저장
                crawl_results[source_id] = activities_data
                found_count += 1
                processed_in_page += 1
                print(f"    ✅ 크롤링 완료: {len(activities_data)}개 결재선")
                
                # 임시 결과 저장 (10개마다)
                if found_count % 10 == 0:
                    with open(CRAWL_RESULT_JSON, 'w', encoding='utf-8') as f:
                        json.dump(crawl_results, f, ensure_ascii=False, indent=2)
                    print(f"    💾 임시 저장: {found_count}개 완료")
                
                # 뒤로가기
                driver.back()
                time.sleep(1)
                
                # 그리드 프레임으로 복귀
                switch_into_frame_having(GRID)
                time.sleep(0.5)
                
                # 다음 인덱스로
                idx += 1
                
            except Exception as e:
                print(f"    ❌ 에러: {e}")
                error_count += 1
                
                # 복구 시도
                try:
                    driver.back()
                    time.sleep(1)
                    switch_into_frame_having(GRID)
                except:
                    pass
                
                idx += 1
                continue
        
        print(f"  📊 이 페이지에서 {processed_in_page}개 문서 크롤링 완료")
        
        # 모든 대상을 찾았으면 종료
        if len(crawl_results) >= len(need_crawling):
            print(f"\n✅ 모든 문서 크롤링 완료!")
            break
        
        # 다음 페이지 확인 및 이동
        if has_next_page():
            print(f"  📄 다음 페이지로 이동 중...")
            if go_to_next_page():
                page_num += 1
            else:
                print(f"  ⚠️ 페이지 이동 실패, 종료")
                break
        else:
            print(f"\n⚠️ 다음 페이지 없음, 종료")
            break

    # 최종 결과 저장
    with open(CRAWL_RESULT_JSON, 'w', encoding='utf-8') as f:
        json.dump(crawl_results, f, ensure_ascii=False, indent=2)
    
    print("\n" + "="*70)
    print("📊 크롤링 결과:")
    print(f"  - 성공: {found_count}개")
    print(f"  - 에러: {error_count}개")
    print(f"  - 결과 저장: {CRAWL_RESULT_JSON}")
    print("="*70)

finally:
    driver.quit()

# ─────────────────────────────────────────────────────────
# 7) cmds 파일 업데이트
# ─────────────────────────────────────────────────────────
print("\n📝 cmds 파일 업데이트 중...")

if not Path(CMDS_INPUT).exists():
    print(f"❌ {CMDS_INPUT} 파일이 없습니다!")
    exit(1)

updated_count = 0
total_lines = 0

with open(CMDS_INPUT, 'r', encoding='utf-8') as f_in, \
     open(CMDS_OUTPUT, 'w', encoding='utf-8') as f_out:
    
    for line in f_in:
        total_lines += 1
        line = line.strip()
        
        if not line.startswith("addDocument "):
            f_out.write(line + '\n')
            continue
        
        # JSON 파싱
        try:
            json_str = line[12:]  # "addDocument " 제거
            doc = json.loads(json_str)
            source_id = doc.get("sourceId")
            
            # 크롤링 결과에 있는지 확인
            if source_id not in crawl_results:
                f_out.write(line + '\n')
                continue
            
            # activities 업데이트
            crawled = crawl_results[source_id]
            need_update = need_crawling.get(source_id, [])
            activities = doc.get("activities", [])
            
            for update_info in need_update:
                target_index = update_info["index"]
                target_name = update_info["name"]
                target_action = update_info["action_type"]
                
                # activities에서 매칭되는 항목 찾기
                if target_index < len(activities):
                    activity = activities[target_index]
                    
                    # index, name, action_type 모두 확인
                    activity_name = activity.get("name", "")
                    activity_type = activity.get("type", "")
                    
                    if target_name in activity_name and target_action == activity_type:
                        # 크롤링 결과에서 해당 항목 찾기
                        for crawled_item in crawled:
                            if (crawled_item["index"] == target_index and 
                                crawled_item["name"] in activity_name and
                                crawled_item["action_type"] == activity_type):
                                
                                # actionDate 업데이트
                                old_date = activity.get("actionDate")
                                activity["actionDate"] = crawled_item["timestamp"]
                                print(f"  ✅ {source_id} - {target_name}: {old_date} → {crawled_item['timestamp']}")
                                updated_count += 1
                                break
            
            # 업데이트된 문서 저장
            updated_line = "addDocument " + json.dumps(doc, ensure_ascii=False, separators=(',', ':'))
            f_out.write(updated_line + '\n')
            
        except Exception as e:
            print(f"  ⚠️ 파싱 에러 (line {total_lines}): {e}")
            f_out.write(line + '\n')

print("\n" + "="*70)
print("🎉 업데이트 완료!")
print("="*70)
print(f"📊 처리 결과:")
print(f"  - 전체 문서: {total_lines}개")
print(f"  - 업데이트: {updated_count}개 결재선")
print(f"  - 출력 파일: {CMDS_OUTPUT}")
print("="*70)

🚀 결재 시간 크롤링 시작
✅ need_crawling.json 로드 완료: 171개 문서
📋 크롤링 대상: 171개 문서

✅ 기존 크롤링 결과 로드: 0개
📋 남은 크롤링 대상: 171개



✋ 로그인 완료 후 Enter를 눌러주세요...  



📂 전자결재 메뉴 진입 중...
📂 결재 문서관리 진입 중...

⚙️ 필터 설정 중...
✓ 요소 발견 (프레임 1)
  - 상태: 완료
  - 기간: 2016-01-01 ~ 2019-12-31

🔍 조회 버튼 클릭...

📊 그리드 데이터 크롤링 중...
✓ 요소 발견 (프레임 1)

📄 페이지 1 처리 중...
  - 현재 페이지 문서 수: 50개
  📊 이 페이지에서 0개 문서 크롤링 완료
  📄 다음 페이지로 이동 중...

📄 페이지 2 처리 중...
  - 현재 페이지 문서 수: 50개
  📊 이 페이지에서 0개 문서 크롤링 완료
  📄 다음 페이지로 이동 중...

📄 페이지 3 처리 중...
  - 현재 페이지 문서 수: 50개
  📊 이 페이지에서 0개 문서 크롤링 완료
  📄 다음 페이지로 이동 중...

📄 페이지 4 처리 중...
  - 현재 페이지 문서 수: 50개
  📊 이 페이지에서 0개 문서 크롤링 완료
  📄 다음 페이지로 이동 중...

📄 페이지 5 처리 중...
  - 현재 페이지 문서 수: 50개
  📊 이 페이지에서 0개 문서 크롤링 완료
  📄 다음 페이지로 이동 중...

📄 페이지 6 처리 중...
  - 현재 페이지 문서 수: 50개
  📊 이 페이지에서 0개 문서 크롤링 완료
  📄 다음 페이지로 이동 중...

📄 페이지 7 처리 중...
  - 현재 페이지 문서 수: 50개
  📊 이 페이지에서 0개 문서 크롤링 완료
  📄 다음 페이지로 이동 중...

📄 페이지 8 처리 중...
  - 현재 페이지 문서 수: 50개
  📊 이 페이지에서 0개 문서 크롤링 완료
  📄 다음 페이지로 이동 중...

📄 페이지 9 처리 중...
  - 현재 페이지 문서 수: 50개
  📊 이 페이지에서 0개 문서 크롤링 완료
  📄 다음 페이지로 이동 중...

📄 페이지 10 처리 중...
  - 현재 페이지 문서 수: 50개
  📊 이 페이지에서 0개 문서 크롤링 완료
  📄 다음 페이지로 이동 중...

📄 페이지 